In [1]:
import os

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "add-your-langsmith-key"  # Your API key

os.environ['REDIS_SCHEMA'] = "schema_lcdocs_dim_768.yml"

In [2]:
import uuid

# Generate a unique run ID for this experiment
run_uid = uuid.uuid4().hex[:6]

In [3]:
from langchain_benchmarks import clone_public_dataset, registry

registry = registry.filter(Type="RetrievalTask")
registry

Name,Type,Dataset ID,Description
LangChain Docs Q&A,RetrievalTask,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Semi-structured Reports,RetrievalTask,c47d9617-ab99-4d6e-a6e6-92b8daf85a7d,Questions and answers based on PDFs containing tables and charts. The task provides the raw documents as well as factory methods to easily index them and create a retriever. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Multi-modal slide decks,RetrievalTask,40afc8e7-9d7e-44ed-8971-2cae1eb59731,This public dataset is a work-in-progress and will be extended over time. Questions and answers based on slide decks containing visual tables and charts. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer.


In [4]:
langchain_docs = registry["LangChain Docs Q&A"]
langchain_docs

Name,LangChain Docs Q&A
Type,RetrievalTask
Dataset ID,452ccafc-18e1-4314-885b-edd735f17b9d
Description,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,conversational-retrieval-qa
get_docs,


In [5]:
clone_public_dataset(langchain_docs.dataset_id, dataset_name=langchain_docs.name)

Dataset LangChain Docs Q&A already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/9534e90b-1d2b-55ed-bf79-31dc5ff16722/datasets/3ce3b4a1-0640-4fbf-925e-2c03caceb5ac.


In [6]:
docs = list(langchain_docs.get_docs())
print(repr(docs[0])[:100] + "...")

File /usr/local/lib/python3.11/site-packages/langchain_benchmarks/rag/tasks/langchain_docs/indexing/db_docs/docs.parquet does not exist. Downloading from GCS...
File https://storage.googleapis.com/benchmarks-artifacts/langchain-docs-benchmarking/docs.parquet downloaded.
Document(page_content="LangChain cookbook | 🦜️🔗 Langchain\n\n[Skip to main content](#docusaurus_skip...


In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Redis
from rag_redis.config import EMBED_MODEL, INDEX_NAME, INDEX_SCHEMA, REDIS_URL

In [8]:
embedder = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

_ = Redis.from_texts(
    # appending this little bit can sometimes help with semantic retrieval
    # especially with multiple companies
    texts=[d.page_content for d in docs],
    metadatas=[d.metadata for d in docs],
    embedding=embedder,
    index_name=INDEX_NAME,
    index_schema=INDEX_SCHEMA,
    redis_url=REDIS_URL,
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

`index_schema` does not match generated metadata schema.
If you meant to manually override the schema, please ignore this message.
index_schema: {'text': [{'name': 'content'}, {'name': 'changefreq'}, {'name': 'description'}, {'name': 'language'}, {'name': 'loc'}, {'name': 'priority'}, {'name': 'source'}, {'name': 'title'}], 'vector': [{'name': 'content_vector', 'algorithm': 'HNSW', 'datatype': 'FLOAT32', 'dims': 768, 'distance_metric': 'COSINE'}]}
generated_schema: {'text': [{'name': 'changefreq'}, {'name': 'description'}, {'name': 'language'}, {'name': 'loc'}, {'name': 'priority'}, {'name': 'source'}, {'name': 'title'}], 'numeric': [], 'tag': []}



In [9]:
vectorstore = Redis.from_existing_index(
    embedding=embedder, index_name=INDEX_NAME, schema=INDEX_SCHEMA, redis_url=REDIS_URL
)
retriever = vectorstore.as_retriever(k=6)

In [10]:
from operator import itemgetter
from typing import Sequence

#from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable.passthrough import RunnableAssign


# After the retriever fetches documents, this
# function formats them in a string to present for the LLM
def format_docs(docs: Sequence[Document]) -> str:
    formatted_docs = []
    for i, doc in enumerate(docs):
        doc_string = (
            f"<document index='{i}'>\n"
            f"<source>{doc.metadata.get('source')}</source>\n"
            f"<doc_content>{doc.page_content}</doc_content>\n"
            "</document>"
        )
        formatted_docs.append(doc_string)
    formatted_str = "\n".join(formatted_docs)
    return f"<documents>\n{formatted_str}\n</documents>"


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant answering questions about LangChain."
            "\n{context}\n"
            "Respond solely based on the document content.",
        ),
        ("human", "{question}"),
    ]
)
#llm = ChatAnthropic(model="claude-2.1", temperature=1)
llm = Ollama(model="llama2")

response_generator = (prompt | llm | StrOutputParser()).with_config(
    run_name="GenerateResponse",
)

# This is the final response chain.
# It fetches the "question" key from the input dict,
# passes it to the retriever, then formats as a string.

chain = (
    RunnableAssign(
        {
            "context": (itemgetter("question") | retriever | format_docs).with_config(
                run_name="FormatDocs"
            )
        }
    )
    # The "RunnableAssign" above returns a dict with keys
    # question (from the original input) and
    # context: the string-formatted docs.
    # This is passed to the response_generator above
    | response_generator
)

In [12]:
chain.invoke({"question": "What's expression language?"})

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


'Hello! Expression Language (EL) is a programming language used for building and defining the logic of LLM Chains, which are a type of neural network model that can be used for natural language processing tasks like text classification, generation, and question-answering. EL allows developers to define and chain together multiple steps of computation using a simple and intuitive syntax, making it easy to create complex models with many inputs, outputs, and conditions.\n\nIn the context of LLM Chains, Expression Language is used to define the sequence of computations that make up each chain, including the inputs, outputs, and any necessary transformations or operations. This allows developers to easily create and manipulate chains for a wide range of tasks, from simple text classification to complex question-answering systems.\n\nSome key features of Expression Language include:\n\n* A simple syntax for defining computations, with support for many common programming constructs like cond

In [13]:
from langchain_benchmarks.rag import get_eval_config
from langchain_benchmarks.utils import run_without_langsmith

In [14]:
#RAG_EVALUATION = get_eval_config()
#test_run = run_without_langsmith(
#        # This will clone the dataset locally if not already there
#        path_or_token_id=langchain_docs.dataset_id,
#        llm_or_chain_factory=chain,
#        evaluation=None,
#        verbose=True,
#    )

In [15]:
from langsmith.client import Client
from langchain_benchmarks.rag import get_eval_config

In [16]:
client = Client()
#RAG_EVALUATION = get_eval_config()

test_run = client.run_on_dataset(
    dataset_name=langchain_docs.name,
    llm_or_chain_factory=chain,
    evaluation=None,
    project_name=f"llama-2 xeon-qa-chain simple-index {run_uid}",
    project_metadata={
        "index_method": "basic",
    },
    verbose=True,
)

View the evaluation results for project 'llama-2 xeon-qa-chain simple-index 435397' at:
https://smith.langchain.com/o/9534e90b-1d2b-55ed-bf79-31dc5ff16722/datasets/3ce3b4a1-0640-4fbf-925e-2c03caceb5ac/compare?selectedSessions=67e02009-fc07-4612-983f-cf216fd96fde

View all tests for Dataset LangChain Docs Q&A at:
https://smith.langchain.com/o/9534e90b-1d2b-55ed-bf79-31dc5ff16722/datasets/3ce3b4a1-0640-4fbf-925e-2c03caceb5ac
[>                                                 ] 0/86

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


[>                                                 ] 1/86

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.
score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


[>                                                 ] 2/86

...

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


[---------------------------------------------->   ] 80/86

score_threshold is deprecated. Use distance_threshold instead.score_threshold should only be used in similarity_search_with_relevance_scores.score_threshold will be removed in a future release.


[------------------------------------------------->] 86/86

,error,execution_time,run_id
count,0,86.000000,86
unique,0,NaN,86
top,NaN,NaN,6797bc09-b460-4b24-8b63-28aa7c63ddf0
freq,NaN,NaN,1
mean,NaN,196.718944,NaN
std,NaN,35.497965,NaN
min,NaN,30.937834,NaN
25%,NaN,179.980567,NaN
50%,NaN,200.070071,NaN
75%,NaN,217.983161,NaN
